<a href="https://colab.research.google.com/github/SofJRL/AIMaster_ActivityModule2/blob/main/ML_identifySCDsubjects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_curve, auc, classification_report
from imblearn.over_sampling import SMOTE
import plotly.express as px

# Step 2: Upload dataset
from google.colab import files
uploaded = files.upload()  # Upload the dataset file

df = pd.read_csv("DATASET_Activity2.csv", sep=";")  # Use semicolon as separator
print("Data loaded:")
print(df.head())

# Paso 3: Exploración inicial
print("\nInformación del dataset:")
print(df.info())
print("\nEstadísticas descriptivas:")
print(df.describe())

# Paso 4: Visualización de las variables
# Distribuciones de variables numéricas
num_vars = ['Age', 'MMSE', 'GDS']
for col in num_vars:
    # Check if column exists in DataFrame before plotting
    if col in df.columns:
        plt.figure(figsize=(8, 5))
        sns.histplot(df[col], kde=True, bins=15, color="skyblue")
        plt.title(f"Distribución de {col}")
        plt.show()
    else:
        print(f"Column '{col}' not found in DataFrame. Skipping...")

# Distribución de la variable objetivo (Conversion)
plt.figure(figsize=(6, 4))
sns.countplot(x='Conversion', data=df, palette='pastel')
plt.title("Distribución de la variable objetivo")
plt.show()

# Paso 5: Balancear las clases
print("\nDistribución antes del balanceo:")
print(df['Conversion'].value_counts())

# Aplicar SMOTE
X = df.drop(columns=['Conversion', 'ID'])  # Elimina la variable objetivo e ID
y = df['Conversion']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("\nDistribución después del balanceo:")
print(pd.Series(y_resampled).value_counts())

# Paso 6: Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Paso 7: Entrenar un modelo Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Predicción y evaluación inicial
y_pred = rf.predict(X_test)

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# Paso 8: Optimización de hiperparámetros con GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("\nMejores hiperparámetros:", grid_search.best_params_)

# Entrenar con los mejores parámetros
best_rf = grid_search.best_estimator_

# Paso 9: Identificar las variables más importantes
importances = best_rf.feature_importances_
features = X.columns
importance_df = pd.DataFrame({'Feature': features, 'Importance': importances}).sort_values(by='Importance', ascending=False)

# Gráfico de barras para la importancia de características
fig = px.bar(importance_df, x='Importance', y='Feature', title='Importancia de Variables', orientation='h')
fig.show()

# Paso 10: Evaluación del modelo
# Matriz de confusión
conf_matrix = confusion_matrix(y_test, best_rf.predict(X_test))
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1])
plt.title("Matriz de confusión")
plt.ylabel("Clase verdadera")
plt.xlabel("Clase predicha")
plt.show()

# Curva ROC
y_prob = best_rf.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}', color='darkorange')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()

# Paso 11: Resumen interactivo
# Gráfico ROC interactivo con Plotly
roc_data = pd.DataFrame({'FPR': fpr, 'TPR': tpr, 'Threshold': thresholds})
fig_roc = px.area(roc_data, x='FPR', y='TPR', title=f'Curva ROC (AUC = {roc_auc:.2f})',
                  labels={'FPR': 'Tasa de Falsos Positivos', 'TPR': 'Tasa de Verdaderos Positivos'})
fig_roc.show()


Saving DATASET_Activity2.csv to DATASET_Activity2 (3).csv
Data loaded:
  ID;Conversion;Age;Gender;Education;HTA;DLP;Smoker;Antidepressants;MMSE;GDS
0                         0001;1;73;1;2;1;1;1;0;29;5                        
1                        0002;0;74;1;2;1;1;0;0;25;21                        
2                        0003;0;77;0;3;1;0;0;0;27;13                        
3                         0004;0;53;0;3;1;1;0;0;30;5                        
4                        0005;0;60;0;2;0;1;2;0;29;14                        

Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 1 columns):
 #   Column                                                                      Non-Null Count  Dtype 
---  ------                                                                      --------------  ----- 
 0   ID;Conversion;Age;Gender;Education;HTA;DLP;Smoker;Antidepressants;MMSE;GDS  99 non-null     object
dtypes: object(1)
memory usage:

KeyError: 'Age'

<Figure size 800x500 with 0 Axes>